# Exploratory Data Analysis

This section investigates patterns and relationships among the vessels that did not appear in the IMO database. 

## Data Dictionary

| Column | Datatype | Description |
|---------------------------|----------|-------------------------------------------------------------------------------------------------------|
| ssvid | int64 | the vessel's MMSI code which is it's "unique" AIS ID |
| gap_hours | float64 | length of the gap event |
| gap_distance_m | float64 | distance of the gap event in miles |
| gap_implied_speed_knots | float64 |  |
| positions_per_day | float64 |  |
| vessel_class | object | vessel geartype |
| flag | object | The state a vessel is registered or licensed under |
| off_timestamp | object | timestamp AIS was turned off |
| off_msgid | object | message ID from AIS turning off |
| off_lat | float64 | latitude when AIS turned off |
| off_lon | float64 | longitude when AIS turned off |
| off_type | object | the class of AIS device (A or B), Class A devices are more expensive, have stronger signals and broadcast more frequently |
| off_receiver_type | object | whether the AIS message was recieved by a satellite or terrestrial receiver when turned off |
| off_distance_from_shore_m | float64 | distance from shore when AIS turned off |
| on_timestamp | object | timestamp AIS was turned on |
| on_msgid | object | message ID from AIS turning on |
| on_lat | float64 | latitude when AIS turned on |
| on_lon | float64 | longitude when AIS turned on |
| on_type | object | the class of AIS device (A or B), class A devices have stronger signals and broadcast more frequently |
| on_receiver_type | object | whether the AIS message was recieved by a satellite or terrestrial receiver when turned on |
| on_distance_from_shore_m | float64 | distance from shore when AIS turned on |

## Import Packages and Data

In [ ]:
# standard data manipulation libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Clustering models
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

In [ ]:
# import IMO non registered list and gap events dataset
imo_notreg = pd.read_csv('./data/imo_notreg.csv')
gap_events = pd.read_csv('./data/raw_sample.csv') 

In [ ]:
# check the contents
print(f'Total Non-Registered Vessels: {len(imo_notreg)}')
print(f'Gap Events: {len(gap_events)}')

In [ ]:
imo_notreg['0'].nunique()

In [ ]:
# Confirm highest number of times a vessel appears
imo_notreg['0'].value_counts()

## Combine and Clean Datasets

In [ ]:
# Filter the full gap events dataset to only include the non registered vessels
gap_notreg = gap_events[gap_events['ssvid'].isin(imo_notreg['0'])]
gap_notreg.shape

In [ ]:
# change ssvid column name for consistency
gap_notreg = gap_notreg.rename(columns={'ssvid':'mmsi'})
gap_notreg.head()

In [ ]:
gap_notreg.describe()

In [ ]:
# Check which countries have the most gap events?

plt.figure(figsize=(18,6))
countries = sns.countplot(x="flag", data=gap_notreg, palette="Set3",
              order=gap_notreg['flag'].value_counts().iloc[:20].index).set(xlabel='Country',
                                                                           ylabel='Observations',
                                                                           title='Count of Gap Events by Country (top 20)');

# for bar in countries.patches:
#     countries.annotat(format(bar.get_height(), '.2f'), 
#                       (p.get_x() + p.get_width() / 2., 
#                        p.get_height()), 
#                        ha = 'center',  
#                        va = 'center', 
#                        xytext = (0, 10), 
#                        textcoords = 'offset points')
    
    

In [ ]:
# Check for duplicate rows
duplicates = gap_notreg.duplicated()
gap_notreg[duplicates].head()

In [ ]:
# Drop duplicate rows
gap_notreg = gap_notreg.drop_duplicates()

In [ ]:
# Change time columns to datetime
gap_notreg['off_timestamp'] = pd.to_datetime(gap_notreg['off_timestamp'], format='%Y-%m-%d %H:%M:%S')
gap_notreg['on_timestamp'] = pd.to_datetime(gap_notreg['on_timestamp'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
# Check timeframe
gap_notreg['off_timestamp'].min(), gap_notreg['off_timestamp'].max()

# DBSCAN Model: Clustering Length of Gap Event

In addition to looking at a vessel's location at the point of going dark, it's worth taking into account the length of the gap event. This DBSCAN model also takes in the gap_hours variable. Increasing the number of variables proved to be very computationally expensive, so I use principle component analysis for dimensionality reduction.


|  eps  | min_samples | number of clusters | PCA |
|:-----:|:-----------:|:------------------:|:---:|
|  0.05 |      40     |         66         |  Y  |
|  0.05 |      40     |         66         |  N  |
| 0.001 |      40     |         294        |  Y  | 

In [ ]:
20 / 6371.0088

In [ ]:
# Create dataframe of only lat/lon/gap_hr data
X = gap_notreg[['off_lat', 'off_lon', 'gap_hours']]
X.shape

In [ ]:
# Scale the data
ss = StandardScaler()
X_sc = ss.fit_transform(X)

In [ ]:
# Dimensionality reduction with PCA
pca = PCA(n_components= X_sc.shape[1])
X_pca = pca.fit_transform(X_sc)

In [ ]:
# Instantiate and fit principle components
dbscan = DBSCAN(eps=0.05, min_samples= 40, metric='euclidean')
dbscan.fit(X_pca)

In [ ]:
# Instantiate and fit
dbscan = DBSCAN(eps=.05, min_samples= 40, metric='euclidean')
dbscan.fit(X_sc)

In [ ]:
# check how many clusters were created
# len(set(dbscan.labels_)) 

In [ ]:
# Check the silhouette score
# silhouette_score(off_coords, dbscan.labels_)

# DBSCAN Model: Clustering Coordinates Where Vessels Go Dark

The popular clustering algorithm DBSCAN is a useful tool for exploration at this stage because it creates clusters by linking nearby data points to one another. I will feed in the location data at the moment vessels go dark (turn off their AIS transponders).

The latitude and longitude coordinates must be [converted from degrees to radians](https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/) in order to use Scikit-Learn's haversine distance metric. The algorithm uses an epsilon value (distance threshold) of 20 km, which is also converted to radian units. Min samples are set at 40 as it seemed reasonable to assume that 40 instances of a vessel going dark within a 20 km radius is indicative of suspicious behavior. Scaling is not required since the data is in latitude longitude coordinates. **Add comment on ball tree algorithm**

## Prepare Latitude and Longitude Data

In [ ]:
# Make a dataframe of only lat/lon at AIS switch off

latlon_off = gap_notreg[['off_lat', 'off_lon']]
latlon_on = gap_notreg[['on_lat', 'on_lon']]

latlon_off.head()

In [ ]:
latlon_on.head()

In [ ]:
# convert columns to numpy matrices
off_coords = latlon_off.to_numpy()
on_coords = latlon_on.to_numpy()

# check array
off_coords

In [ ]:
# convert epsilon and coordinates to radians 
# code adapted from Geoff Boeing
kms_per_radian = 6371.0088
epsilon = 20 / kms_per_radian 

off_coords = np.radians(off_coords)
on_coords = np.radians(on_coords)

# check array
off_coords

## Fit and Evaluate

In [ ]:
# instantiate and fit
dbscan = DBSCAN(eps=epsilon, min_samples= 40, algorithm='ball_tree', metric='haversine')
dbscan.fit(off_coords)

In [ ]:
# check how many clusters were created
len(set(dbscan.labels_)) 

In [ ]:
# pass in data and clusters to get the silhouette score
#silhouette_score(off_coords, dbscan.labels_)

In [ ]:
# Create cluster column
latlon_off['off_cluster'] = dbscan.labels_
latlon_off.head()

In [ ]:
# Which clusters have the most observations?
latlon_off['off_cluster'].value_counts()

## Visualize Model Results

In [ ]:
plt.figure(figsize=(15, 15))
plt.scatter(latlon_off['off_lat'], latlon_off['off_lon'], c=dbscan.labels_, s=1, cmap="tab20")
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Location of Vessel at AIS Off');

In [ ]:
# Break top 3 clusters into dataframes and visualize (including noise)

cluster0 = latlon_off[latlon_off['off_cluster'] == 0]
cluster_1 = latlon_off[latlon_off['off_cluster'] == -1]
cluster11 = latlon_off[latlon_off['off_cluster'] == 11]
cluster18 = latlon_off[latlon_off['off_cluster'] == 18]

plt.figure(figsize=(15, 15))

plt.scatter(cluster0['off_lat'], cluster0['off_lon'], s=1, c='skyblue');
plt.scatter(cluster_1['off_lat'], cluster_1['off_lon'], s=1, c='lavender');
plt.scatter(cluster11['off_lat'], cluster11['off_lon'], s=1, c='b')
plt.scatter(cluster18['off_lat'], cluster18['off_lon'], s=1, c='gold')
plt.title('Largest Clusters: Location at AIS Off')
plt.xlabel('Latitude')
plt.ylabel('Longitude');

## Add Predictions to Dataset

In [ ]:
# Merge cluster into the original dataframe
gap_notreg = gap_notreg.merge(latlon_off, how='outer', left_index=True, right_index=True)

# Drop the additional lat/lon columns
gap_notreg.drop(columns=['off_lat_y', 'off_lon_y'], inplace=True)

In [ ]:
# Rename columns
gap_notreg.rename(columns={'off_lat_x' : 'off_lat',
                           'off_lon_x' : 'off_lon'}, inplace=True)

In [ ]:
# Save updated dataframe to CSV
gap_notreg.to_csv('./data/gap_notreg.csv', inplace=False)

In [ ]:
# Explore any patterns in the first few clusters
gap_notreg.groupby('off_cluster')[['gap_hours', 
                                   'gap_distance_m',
                                   'positions_per_day', 
                                   'on_distance_from_shore_m',
                                   'off_distance_from_shore_m',]].mean().T.loc[:,[0,11,18]]

In [ ]:
# Break into individual dataframes
cluster0 = gap_notreg[gap_notreg['off_cluster'] == 0]
cluster11 = gap_notreg[gap_notreg['off_cluster'] == 11]
cluster18 = gap_notreg[gap_notreg['off_cluster'] == 18]

# EDA on Location Clusters

- Total Observations: 208,866
- Unique Vessels (MMSI): 30,297
- Avg Length of Gaps: 148.70 hrs
- Avg Num Positions per Day: 9.57

In [ ]:
# Which vessel classes appear the most in cluster 0 and how often?

plt.figure(figsize=(12,9)) 

plt.subplot(2,2,1)
sns.countplot(y="vessel_class", 
              data=cluster0, 
              palette="icefire_r",
              order=cluster0['vessel_class'].value_counts().iloc[:10].index).set(title=('Cluster 0 Vessel Classes'),
                                                                                 xlabel=('Vessel Count'),
                                                                                 ylabel=('Gear Type'))
plt.subplot(2,2,2)
sns.countplot(y="vessel_class", 
              data=cluster11, 
              palette="icefire_r",
              order=cluster11['vessel_class'].value_counts().iloc[:10].index).set(title=('Cluster 11 Vessel Classes'),
                                                                                 xlabel=('Vessel Count'),
                                                                                 ylabel=('Gear Type'))
plt.subplot(2,2,3)
sns.countplot(y="vessel_class", 
              data=cluster18, 
              palette="icefire_r",
              order=cluster18['vessel_class'].value_counts().iloc[:10].index).set(title=('Cluster 18 Vessel Classes'),
                                                                                 xlabel=('Vessel Count'),
                                                                                 ylabel=('Gear Type'))
plt.tight_layout();

In [ ]:
# what is the average gap hours for the vessels classes that appear the most?

cluster0.loc[cluster0['vessel_class'] == 'trawlers']['gap_hours'].mean()
cluster0[(cluster0['vessel_class'] == 'trawlers')]['gap_hours'].mean()

cluster11[(cluster11['vessel_class'] == 'set_longlines')]['gap_hours'].mean()

cluster18[(cluster18['vessel_class'] == 'set_longlines')]['gap_hours'].mean()

In [ ]:
# what is the average number of positions?


In [ ]:
# Which countries have the highest average gaps
cluster0.groupby('flag')['gap_hours'].mean().sort_values(ascending=False)

In [ ]:
# Which vessels have the highest avg gaps and where are they from?
cluster0.groupby(['mmsi','flag','vessel_class'])['gap_hours'].mean().sort_values(ascending=False)

In [ ]:
# Are there any relationships between location and the length of the gap event?
# Within cluster 0 zoom in on the gaps lasting less 1000 hours

def gaps_scatter(cluster):
    low_gaps = cluster[cluster['gap_hours'] < 1000]
    return low_gaps.plot(kind="scatter", 
                  x="off_lat",
                  y="off_lon", 
                  c="gap_hours",
                  cmap="icefire_r", 
                  figsize=(14, 10), 
                  s=4, 
                  title='Length of Gap Events by Location')

In [ ]:
gaps_scatter(cluster0);

In [ ]:
gaps_scatter(cluster11);

In [ ]:
gaps_scatter(cluster18);

In [ ]:
# When did the highest number of gap events take place?


In [ ]:
# check the distribution of gap hours

# high_gaps = gap_notreg[gap_notreg['gap_hours'] > 2000]
# low_gaps = gap_notreg[gap_notreg['gap_hours'] < 2000]

# plt.figure(figsize=(18,6))
# sns.distplot(low_gaps['gap_hours']).set(xlabel='Number of Hours off Radar', 
#                                         ylabel='Number of Vessels', 
#                                         title='Gap Hr Distribution (Under 2000 hrs)');

In [ ]:
# save to csv
gap_notreg.to_csv('./data/gap_notreg.csv', index=False)

# References
- https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/
- https://link.springer.com/chapter/10.1007/978-3-030-38081-6_2